# Webscraping

---

## Scrapen van een website met Selenium

Scrapen met [Selenium] is niet de meest betrouwbare manier van webscraping.  
Maar sommige websites hebben (veel) Javascript of [iframe] HTML tags of laden dynamisch nieuwe elementen waardoor de data moeilijk te scrapen is met andere technieken.  
Het is dan toch mogenlijk om de website te scrapen met Selenium.

---


https://sroze.github.io/ngInfiniteScroll/demo_async.html is een demo website voor een [infinite scroll] Javascript module die werkt met het webapplicatieframework [AngularJS].  
De website gebruikt de [Reddit API] om tot en met 1000 artikelen te laden als de einde van de pagina wordt bereikt.  
In de notebook cells hieronder open we de demo website en gebruiken we Selenium (met een beetje Javascript) om te scrollen.
De artikelen parsen 



[iframe]: https://www.w3docs.com/learn-html/html-iframe-tag.html
[Selenium]: https://pypi.org/project/selenium/
[webdriver_manager]: https://pypi.org/project/webdriver-manager/ 

[infinite scroll]: https://github.com/sroze/ngInfiniteScroll
[AngularJS]: https://angularjs.org/

[Reddit API]: https://www.reddit.com/dev/api


In [1]:
!python -m pip install --upgrade pip selenium webdriver-manager beautifulsoup4

In [2]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

In [3]:
# driver = webdriver.Chrome(ChromeDriverManager().install())  # chrome
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())  # chromium
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())  # firefox

# maak de browser fullscreen
driver.maximize_window()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current chromium version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/home/username/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache


Er browser die geopend is bij het aanroepen van de `webdriver.Chrome`.   
Deze browser wordt nu gecommandeerd door Python via het `driver` object.  

Selenium heeft geen commando om de browser te scrollen maar Javascript wel.  
Selenium kan javascript injecteren in de browser die gestart is door Selenium.

Hieronder een functie die de viewport van de browser naar beneden laat scrollen.  

In [4]:
def scroll_to_end(driver: WebDriver):
    """use javascript to scroll to the bottom of the website"""
    code = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(code)

Door een URL te geven aan het `get` functie van het `webdriver` object wordt er een webpagina geopend in de browser.  

In [5]:
url = 'https://sroze.github.io/ngInfiniteScroll/demo_async.html'
driver.get(url)

De `driver` heeft de browser de url laten openen.  
Nu kan de scroll functie geprobeerd worden.

In [6]:
scroll_to_end(driver)

In de developer tools onder de tab Elements kan er een [XPath] worden gemaakt om een pad naar `WebElement` te maken.  
De input om dit te doen wordt met de toetsencombinatie `Ctrl + F` geactiveerd. 

Met de functie `find_elements_by_xpath` _(elements meervoud)_ wordt alle webelementen opgevraagd die dat XPath hebben.

[XPath]: https://nl.wikipedia.org/wiki/XPath

In [7]:
xpath_item = "//div[@class='demo-container']/div[contains(@class, 'item')]"
items = None
while not items:
    items = driver.find_elements_by_xpath(xpath_item)

In [8]:
items[0]  # check het eerste object in de lijst met WebElements

<selenium.webdriver.remote.webelement.WebElement (session="17d7864259f83fd4400a9d7a56d39711", element="bc9d4395-d13d-4347-ac10-6bf6ffab109c")>

Zoals hieboven in the output is te zien heeft het `WebElement` een *sessie* en een *element*.  
De sessie is de instantie van de `WebDriver` die de browser bestuurd.  
Het element is een UUID (Universally Unique IDentifier).  

Een `WebElement`  is een object dat wijst naar het HTML [element] wat de `WebDriver` op de huidige site heeft gezien.  
Het `WebElement` object houdt geen data vast.  
Alle atributen en methodes van een `WebElement` sturen een commando op naar de `WebDriver`.  
De `WebDriver` zoekt dan het `WebElement` op en stuurt data terug wat opgevraagd is.

Als een `WebElement` vervalt of is veranderd dan is deze "*Stale*"  
Wordt er dan een methode of attribute van het `WebElement` gebruikt, dan zal er een [StaleElementReferenceException] ge-raised worden.  

[UUID]: https://docs.python.org/3/library/uuid.html
[element]: https://nl.wikipedia.org/wiki/Lijst_van_HTML-elementen
[StaleElementReferenceException]: https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html#selenium.common.exceptions.StaleElementReferenceException

In [9]:
print(f"driver sessie ID: {driver.session_id}")

# print de type van het eerste object in de list: items.
print(f"type: {type(items[0])}")
print(f"representatie: {repr(items[0])}")

# print alle te gebruiken methodes & attributen
# print(*[_ for _ in dir(items[0]) if not _.startswith('_')], sep='\n')

driver sessie ID: 17d7864259f83fd4400a9d7a56d39711
type: <class 'selenium.webdriver.remote.webelement.WebElement'>
representatie: <selenium.webdriver.remote.webelement.WebElement (session="17d7864259f83fd4400a9d7a56d39711", element="bc9d4395-d13d-4347-ac10-6bf6ffab109c")>


Hieronder is er een functie gemaakt die de `WebElement` als argument verwacht.  
Met `BeautifulSoup` wordt dan de HTML geparsed om alle data te verkrijgen.  
De data wordt opgevraagd, opgeschoond en direct in een `dict` geplaatst.  
De `dict` met data wordt na het voltooien van de functie teruggegeven door de functie.  

In [10]:
def get_data(item: WebElement) -> dict:
    assert isinstance(item, WebElement), f'received type: {type(item)!r}'    
    html = item.get_attribute('outerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    result = {
        'score': int(soup.find('span', {'class': 'score'}).text.strip()),  # string to int
        'title': soup.find('span', {'class': 'title'}).text.strip(),
        'url':   soup.find('a').get('href')
    }
    return result

Hieronder de geparste data van het eerste object in de lijst met `WebElement`. 

In [11]:
get_data(items[0])

{'score': 1247,
 'title': 'Match Thread: Qualifier 1 - Delhi Capitals vs Chennai Super Kings',
 'url': 'https://www.reddit.com/r/Cricket/comments/q57llz/match_thread_qualifier_1_delhi_capitals_vs/'}

Alle logica is gecreëerd om de browser te scrollen, data op te vragen en de data te parsen.  
Om alle data op te vragen wordt hieronder een `while` loop gebruikt.  

In [12]:
items = []
while len(items) < 999:
    items = driver.find_elements_by_xpath(xpath_item)
    scroll_to_end(driver)    

check hoe veel objecten de lijst `items` bevat

In [13]:
len(items)

1000

Alle data is verkregen en nu kan hier wat mee worden gedaan.  
Bijvoorbeeld kan het artikel met de hoogste score worden opgevraagd.  

In [14]:
# een generator is een luie functie die data geeft waneer dit wordt opgevraagd.  
item_generator = (get_data(item) for item in items)

# verklaar objecten die gebruikt worden in de for-loop
best_article = {}
max_score = 0

# loop om het artikel met de hoogste score te verkrijgen
for data in item_generator:
    score = data['score']
    if max_score < score:
        max_score = score
        best_article = data

best_article

{'score': 99112,
 'title': 'Man calls his parents while skydiving',
 'url': 'https://v.redd.it/6g0d935fpls71'}

Ook kan er van de verkregen data een CSV-file gemaakt worden.

In [15]:
import csv
from pathlib import Path

# maak het pad naar de CSV-file
this_dir = Path.cwd()
csv_file_path = this_dir.joinpath("artikels.csv")

# vraag de headers op voor in de CSV-file
fieldnames = get_data(items[0]).keys()

# verkrijg en sorteer de data, kleinste score eerst
item_iterator = map(get_data, items)
sorted_iterator = sorted(item_iterator, key=lambda d: d.get('score', 0))

# schrijf de data weg in een CSV-file
with csv_file_path.open('w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(sorted_iterator)

Om te checken of het is gelukt kan CSV-file natuurlijk ook gelezen worden.

In [16]:
with csv_file_path.open('r') as csv_file:
    reader = csv.reader(csv_file)
    for num, row in enumerate(reader, start=1):
        print(f"line {num}: {row}")
        if num >= 5:
            break

line 1: ['score', 'title', 'url']
line 2: ['335', 'Throwback to when I said the mystery person on tattooine was boba fett', 'https://i.redd.it/frnq4dfs6os71.jpg']
line 3: ['350', 'Kein Hate, finde es nur echt bemerkenswert.', 'https://i.redd.it/psxucevf6os71.jpg']
line 4: ['351', 'The feeling of being a grandmother.', 'https://i.imgur.com/ZBvT6Yz.gifv']
line 5: ['373', 'The thought of serious lawyers having to watch hundreds of hours of unaired footage of RHOBH just cracks me up.', 'https://i.redd.it/2gdzzitktns71.jpg']


Als alle data is gescrapped die nodig was kunnen we de browser sluiten.

In [17]:
driver.quit()